# Preprocess Measured Data

As the measured data used for calibration does not quite match with the data simulated by EnergyPlus, we need to make some alterations to fix this. As an example of one of these issues, when data is simulated in EnergyPlus at 12 PM it is labeled as 24:00:00 for that day. The mesured data labels this as 00:00:00 the following day and so we must alter the measured data so that it matches with the EnergyPlus generated date.

In [1]:
import pandas as pd
import datetime

### Step 1: Load Data

In [2]:
# outputDict contains info about what the output is called in the measured data (index 0), what it is called in EnergyPlus
# for hourly measurments (index 1), what it is called in EnergyPlus for monthly measurments (index 2) and a unit conversion
# multiplier (index 3).

outputDict = {
    
                'Electricity': ['Building Electricity in kWh',
                                'Electricity:Facility [J](Hourly)',
                                'Electricity:Facility [J](Monthly)',
                                3600000
                                ],
              
                 'DistrictHeating': ['Building Heat Meter in kWh',
                                     'DistrictHeating:Facility [J](Hourly)',
                                     'DistrictHeating:Facility [J](Monthly)',
                                     3600000
                                    ],
    
                 'Water': ['Building Water in litres',
                           'Water:Facility [m3](Hourly)',
                           'Water:Facility [m3](Monthly)',
                           0.001
                            ]
    
             }

In [3]:
# Can choose year from 2017 - 2019.
yearOfInterest = '2018'
nextYear = str(int(yearOfInterest) + 1)


# Can choose from "Electricity", "DistrictHeating" and "Water".
measuredDataOutputOfInterest = 'DistrictHeating'

In [4]:
measuredDataOutputName = outputDict[measuredDataOutputOfInterest][0]
newHourlyMeasuredDataOutputName = outputDict[measuredDataOutputOfInterest][1]
newMonthlyMeasuredDataOutputName = outputDict[measuredDataOutputOfInterest][2]
multiplier = outputDict[measuredDataOutputOfInterest][3]

In [5]:
measured = f'..\Data\InitialMeasuredData\Building_{measuredDataOutputOfInterest}_Measured{yearOfInterest}.csv'

In [6]:
measured_df = pd.read_csv(measured)

### Step 2: Isolate Correct Frequencies

The measured data is collected for every 15 minute interval for 1 year. However, the data which will be used for calibration will be hourly data and monthly data. Therefore, we must preprocess the data to obtain these frequencies.

In [7]:
def splitTimestampDate(row):
    return row['timestamp'].split(' ')[0]

def splitTimestampTime(row):
    return row['timestamp'].split(' ')[1]

In [8]:
measured_df['date'] = measured_df.apply(lambda row: splitTimestampDate(row), axis=1)
measured_df['time'] = measured_df.apply(lambda row: splitTimestampTime(row), axis=1)

In [9]:
measured_df['date_dt'] = pd.to_datetime(measured_df['date'], format='%d/%m/%Y')
measured_df['time_dt'] = pd.to_datetime(measured_df['time'], format='%H:%M')

In [11]:
measured_df['month'] = measured_df['date_dt'].dt.month
measured_df['hour'] = measured_df['time_dt'].dt.hour

In [12]:
hourlyTBOutput = list(measured_df.groupby(['date', 'hour'], axis=0).sum()[measuredDataOutputName])
monthlyTBOutput = list(measured_df.groupby(['month'], axis=0).sum()[measuredDataOutputName])

In [13]:
hourStart = f'{yearOfInterest}-01-01  01:00:00'
hourStart = datetime.datetime.strptime(hourStart, '%Y-%m-%d %H:%M:%S')

hourEnd = f'{nextYear}-01-01 00:00:00'
hourEnd = datetime.datetime.strptime(hourEnd, '%Y-%m-%d %H:%M:%S')

hours = list(pd.date_range(start=hourStart, end=hourEnd, freq='60Min'))
months = ['January', 'Febuary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November',
          'December']

In [14]:
assert len(hourlyTBOutput) == len(hours)
assert len(monthlyTBOutput) == len(months)

In [15]:
HourlyDF = pd.DataFrame({'timestamp_dt': hours, newHourlyMeasuredDataOutputName: hourlyTBOutput})
MonthlyDF = pd.DataFrame({'Date/Time': months, newMonthlyMeasuredDataOutputName: monthlyTBOutput})

### Step 3: Fix Timestamps

As mentioned previously, we now need to alter the timestamps of the hourly data to match with the data simulated by EnergyPlus.

In [16]:
HourlyDF['Month'] = HourlyDF['timestamp_dt'].dt.month
HourlyDF['Day'] = HourlyDF['timestamp_dt'].dt.day
HourlyDF['Hour'] = HourlyDF['timestamp_dt'].dt.hour
HourlyDF['Minute'] = HourlyDF['timestamp_dt'].dt.minute
HourlyDF['Second'] = HourlyDF['timestamp_dt'].dt.second

In [17]:
def getDateTime(row):
    month = str(row['Month'])
    day = str(row['Day'])
    hour = str(row['Hour'])
    minute = str(row['Minute'])
    second = str(row['Second'])
    
    if int(month) < 10:
        month = '0' + month
        
    if int(day) < 10:
        day = '0' + day
        
    if int(hour) < 10:
        hour = '0' + hour
        
    if int(minute) < 10:
        minute = '0' + minute
        
    if int(second) < 10:
        second = '0' + second
    
    mydatetime = ' ' + month + '/' + day + '  ' + hour + ':' + minute + ':' + second
    
    return mydatetime

In [18]:
HourlyDF['Date/Time'] = HourlyDF.apply(lambda row: getDateTime(row), axis=1)

In [19]:
def fixDateTime(row):
    date = datetime.datetime.strptime(row['Date/Time'], ' %m/%d  %H:%M:%S')
    
    if date.hour == 0:
        yesterday = datetime.timedelta(days=1)
        newDate = date - yesterday
        newDateDay = str(newDate.day)
        newDateMonth = str(newDate.month)
        
        if int(newDateDay) < 10:
            newDateDay = '0' + newDateDay
        
        if int(newDateMonth) < 10:
            newDateMonth = '0' + newDateMonth
        
        return ' ' + newDateMonth + '/' + newDateDay + '  ' + '24:00:00'
    
    else:
        return row['Date/Time']

In [20]:
HourlyDF['Date/Time'] = HourlyDF.apply(lambda row: fixDateTime(row), axis=1)

### Step 4: Convert Output to Correct Units

Measured output is in kWh. EnergyPlus output is in J.

In [21]:
HourlyDF[newHourlyMeasuredDataOutputName] = HourlyDF[newHourlyMeasuredDataOutputName]*multiplier

In [22]:
HourlyDF = HourlyDF[['Date/Time', newHourlyMeasuredDataOutputName]]

In [23]:
MonthlyDF[newMonthlyMeasuredDataOutputName] = MonthlyDF[newMonthlyMeasuredDataOutputName]*multiplier

### Step 5: Create CSV Files

In [24]:
createHourlyDF = False

if createHourlyDF:
    HourlyDF.to_csv(f'..\Data\PreprocessedMeasuredData\Building{yearOfInterest}_{measuredDataOutputOfInterest}_AutotuneData_Hourly.csv', index=None)
    
createMonthlyDF = False

if createMonthlyDF:
    MonthlyDF.to_csv(f'..\Data\PreprocessedMeasuredData\Building{yearOfInterest}_{measuredDataOutputOfInterest}_AutotuneData_Monthly.csv', index=None)